In this notebook, we will learn to implement a very simple SMPC model to reproduce the toy example of the beers.

In [1]:
import random

Let's define the encryption function.

First, we assume to have a finite field to which the secrets and the shares belong. To do that, we define a very large prime number `Q` that acts as a modulus.

Then, we can split a secret into `n_shares` shares by simply extracting `n_shares - 1` random numbers from this field and computing the last share accordingly.

In [2]:
Q = 121639451781281043402593

def encrypt(x, n_shares=2):
    shares = list()
    for i in range(n_shares - 1):
        shares.append(random.randint(0, Q))
    final_share = x - (sum(shares) % Q)
    shares.append(final_share)
    return tuple(shares)

print(encrypt(100, 3))

(12364122492218430541415, 64029782574660301741891, -76393905066878732283206)


The secret can be then reconstructed by summing up the secrets.

In [3]:
def decrypt(shares):
    return sum(shares) % Q

Let's consider now the example of the beers.

Three friends have a secret about the number of beers they drank. Let's put their secrets in a list.

In [4]:
secrets = [100, 200, 300]

Now, we can encrypt their secrets into three shares each and put them into `local_shares`.

In [5]:
local_shares = [[], [], []]

for s in secrets:
    t = encrypt(s, 3)
    for i in range(len(t)):
        local_shares[i].append(t[i])

print('Local shares of the secrets:', local_shares)

Local shares of the secrets: [[64278503187106038135448, 32725522716075533885208, 9469642060266965012837], [61438464969625396583246, 12941317312762433099877, 52035700946539323024715], [-4077516375450391316001, -45666840028837966984885, -61505343006806288037252]]


Each party has now a piece of information about itself and the other two parties. The scheme we built has some homomorphic properties:
* we can sum shares (e.g., if `x_1`, `x_2` is a sharing of `x` and `y_1`, `y_2` is a sharing of `y`, then `x_1 + y_1`, `x_2 + y_2` is a sharing of `x + y`);
* we can sum or multiply shares with non-private values.

In [6]:
local_computation = [sum(u) % Q for u in local_shares]

Now, each party has a share of the sum of the secrets. We can now decrypt the final secret (the sum of the initial secrets) with the function created above.

In [7]:
print('Local shares of the sum of the secrets:', local_computation)
print(decrypt(local_computation) / len(local_computation))

Local shares of the sum of the secrets: [106473667963448537033493, 4776031447646109305245, 10389752370186397064455]
200.0


To implement multiplication between shares, we should use the SPDZ protocol, which assumes the presence of a cryptoprovider that generates a triple of private numbers used to perform the multiplication.

To wrap up all the ideas, we better formalize the previous approach creating classes for private and public values in the case of a secure two-party computation.

In [8]:
class PrivateValue:
    def __init__(self, value, share0=None, share1=None):
        if not value is None:
            share0, share1 = encrypt(value)
        self.share0 = share0
        self.share1 = share1

    def decrypt(self):
        return PublicValue(decrypt((self.share0, self.share1)))

    def __add__(self, other):
        if type(other) is PublicValue:
            share0 = (self.share0 + other.value) % Q
            share1 = self.share1
            return PrivateValue(None, share0, share1)
        elif type(other) is PrivateValue:
            share0 = (self.share0 + other.share0) % Q
            share1 = (self.share1 + other.share1) % Q
            return PrivateValue(None, share0, share1)

    def __sub__(self, other):
        if type(other) is PublicValue:
            share0 = (self.share0 - other.value) % Q
            share1 = self.share1
            return PrivateValue(None, share0, share1)
        elif type(other) is PrivateValue:
            share0 = (self.share0 - other.share0) % Q
            share1 = (self.share1 - other.share1) % Q
            return PrivateValue(None, share0, share1)

    def __mul__(self, other):
        def generate_mul_triple():
            # Generated by the crypto provider
            a = random.randrange(Q)
            b = random.randrange(Q)
            c = (a * b) % Q
            return PrivateValue(a), PrivateValue(b), PrivateValue(c)
            # The shares of these numbers are distributed to the two parts

        if type(other) is PublicValue:
            share0 = (self.share0 * other.value) % Q
            share1 = (self.share1 * other.value) % Q
            return PrivateValue(None, share0, share1)
        elif type(other) is PrivateValue:
            a, b, a_mul_b = generate_mul_triple()
            alpha = (self - a).decrypt()
            # i.e., each party subtracts a from self, then they jointly reconstruct this value
            beta = (other - b).decrypt()
            # i.e., each party subtracts a from other, then they jointly reconstruct this value
            return (alpha * beta) + (a * beta) + (b * alpha) + a_mul_b
            # i.e., (self * other - self * b - other * a + a * b) + (other * a - a * b) +
            # (self * b - a * b) + (a * b)

class PublicValue:
    def __init__(self, value):
        self.value = value

    def __add__(self, other):
        if type(other) is PrivateValue:
            return other + self
        elif type(other) is PublicValue:
            return PublicValue(self.value + other.value)

    def __sub__(self, other):
        if type(other) is PrivateValue:
            return other * PublicValue(-1) + self
        elif type(other) is PublicValue:
            return PublicValue(self.value - other.value)

    def __mul__(self, other):
        if type(other) is PrivateValue:
            return other * self
        elif type(other) is PublicValue:
            return PublicValue(self.value * other.value)

Let's play with this new environment

In [ ]:
print('Public number with value 7:', PublicValue(7).value)
print('Private number with value 7 has the shares:', PrivateValue(7).share0, PrivateValue(7).share1)

What happens with the operations?

In [ ]:
x = PrivateValue(5)
y = PrivateValue(3)
z = PublicValue(10)
s = x * y + z
print('Shares of s:', s.share0, s.share1)
print('Reconstructed value:', s.decrypt().value)